In [3]:
import os

os.makedirs("docs", exist_ok=True)

print("docs/ 目录已准备好。请把所有 .md 文件放在 docs/ 下面（至少要有 main.md）。")
print("当前 docs 目录下的文件：", os.listdir("docs"))


docs/ 目录已准备好。请把所有 .md 文件放在 docs/ 下面（至少要有 main.md）。
当前 docs 目录下的文件： ['main.md', 'music.md', 'podcast.md', 'citywalk.md', 'drama.md', 'variety.md']


In [4]:
import os
import shutil
import re

SRC_DIR = "docs"
OUT_DIR = "gen_docs"

# ================================
# 0. 检查并准备目录
# ================================
if not os.path.isdir(SRC_DIR):
    raise FileNotFoundError("请先创建 docs/ 目录并放入 main.md 等源文件。")

os.makedirs(OUT_DIR, exist_ok=True)

# 清空 gen_docs
for f in os.listdir(OUT_DIR):
    path = os.path.join(OUT_DIR, f)
    if os.path.isdir(path):
        shutil.rmtree(path)
    else:
        os.remove(path)

print("gen_docs/ 已清空并准备好。")

# ================================
# 1. 拷贝 md 文件
# ================================
md_files = [f for f in os.listdir(SRC_DIR) if f.lower().endswith(".md")]
md_files.sort()

if "main.md" not in [f.lower() for f in md_files]:
    raise FileNotFoundError("docs 中必须包含 main.md。")

# 置顶 A Quiet Moments
priority_page = "A Quiet Moments.md"
if priority_page in md_files:
    md_files.remove(priority_page)
    md_files.insert(0, priority_page)

# 拷贝
for f in md_files:
    shutil.copy(os.path.join(SRC_DIR, f), os.path.join(OUT_DIR, f))

print("源 md 文件已复制到 gen_docs/。")

# ================================
# 2. 读取标题
# ================================
titles = {}

for fname in md_files:
    path = os.path.join(OUT_DIR, fname)
    title = None
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            if line.strip().startswith("#"):
                title = line.strip().lstrip("#").strip()
                break
    if not title:
        title = os.path.splitext(fname)[0]
    titles[fname] = title

print("标题读取完毕。")

# ================================
# 3. 自动生成 mkdocs.yml
# ================================
yml = []
yml.append("site_name: Jenny Website")
yml.append("use_directory_urls: false")
yml.append("docs_dir: gen_docs")
yml.append("nav:")

# 左侧导航不包含 main.md
nav_files = [f for f in md_files if f.lower() != "main.md"]

# A Quiet Moments 放最前（已在 md_files 排过）
# 加入 nav
for fname in nav_files:
    yml.append(f"  - {titles[fname]}: {fname}")

yml.append("theme:")
yml.append("  name: material")

with open("mkdocs.yml", "w", encoding="utf-8") as f:
    f.write("\n".join(yml))

print("mkdocs.yml 已生成。")

# ================================
# 4. 自动插入英文链接区
# ================================
AUTO_START = "<!-- AUTO-LINKS-START -->"
AUTO_END = "<!-- AUTO-LINKS-END -->"

pattern = re.compile(r"\n?\s*<!-- AUTO-LINKS-START -->.*?<!-- AUTO-LINKS-END -->", re.S)

for fname in md_files:
    path = os.path.join(OUT_DIR, fname)
    with open(path, "r", encoding="utf-8") as f:
        content = f.read()

    content = pattern.sub("", content).rstrip() + "\n\n"

    if fname.lower() == "main.md":
        # 首页显示“Page Navigation”
        others = [(titles[f], f) for f in md_files if f.lower() != "main.md"]
        links = "\n".join(f"- [{t}]({f})" for t, f in others) or "None"

        auto = (
            f"{AUTO_START}\n\n"
            "---\n\n"
            "## Page Navigation\n\n"
            f"{links}\n\n"
            f"{AUTO_END}\n"
        )
    else:
        others = [(titles[f], f) for f in md_files if f != fname]
        links = "\n".join(f"- [{t}]({f})" for t, f in others)

        auto = (
            f"{AUTO_START}\n\n"
            "---\n\n"
            "[Back to Home](main.md)\n\n"
            "## More Pages\n\n"
            f"{links}\n\n"
            f"{AUTO_END}\n"
        )

    with open(path, "w", encoding="utf-8") as f:
        f.write(content + auto)

print("所有页面链接已插入。")


gen_docs/ 已清空并准备好。
源 md 文件已复制到 gen_docs/。
标题读取完毕。
mkdocs.yml 已生成。
所有页面链接已插入。


In [5]:
import sys, subprocess
result = subprocess.run(
    [sys.executable, "-m", "mkdocs", "build"],
    capture_output=True,
    text=True
)
print("stdout:\n", result.stdout)
print("stderr:\n", result.stderr)


stdout:
 
stderr:
 INFO    -  Cleaning site directory
INFO    -  Building documentation to directory: g:\My Drive\Github_chan\chan\site
INFO    -  The following pages exist in the docs directory, but are not included in the "nav" configuration:
  - main.md
INFO    -  Documentation built in 3.08 seconds



In [6]:
import os
import shutil

src = "site"
dst = "."

for item in os.listdir(src):
    s = os.path.join(src, item)
    d = os.path.join(dst, item)

    # 跳过你手写的 index.html
    if item.lower() == "index.html":
        continue

    if os.path.isdir(d):
        shutil.rmtree(d)
    elif os.path.exists(d):
        os.remove(d)

    if os.path.isdir(s):
        shutil.copytree(s, d)
    else:
        shutil.copy2(s, d)

print("网站文件已复制到根目录（已跳过 index.html）。")


网站文件已复制到根目录（已跳过 index.html）。
